# Overview

Make sure that this notebook is run in the Conda environment setup for this project, as described in [Readme.md](README.md)

This notebook runs the code for performing schooling experiments. 

# Parameters

In [2]:
# Prefix for video files (sumo caps name at length 8)
moviefile_prefix = "SC_RN_BG"

# Address for the Enttec USB device (see above)
hw_address = "/dev/cu.usbserial-EN373474"

# Define IP address of smart switch (see README for how to find at command line)
LED_IP = "192.168.0.104"

# Whether to control the lights and recorder. Set to False when not connected to hardware, or when troubleshooting the code.
control_hw = False

# Project directory name
# proj_dir = 'scaleBNT'
proj_dir = 'miniScaleRN'

# Packages

In [3]:
import numpy as np
import def_runexperiments as re
import multiprocess, os, platform
import def_paths as dp
# import pandas as pd

if control_hw:
        # For DMX Control
        from DMXEnttecPro import Controller

        # This is necessary for using multiprocess on a Mac
        if os.name == "posix":
                multiprocess.set_start_method('spawn')

## Root path
Find the root path for the project, based on existing directories. Add to this, if necessary.

In [4]:
# Matt's laptop
if (platform.system() == 'Darwin') and (os.path.expanduser('~')=='/Users/mmchenry'):
    
    root_path = '/Users/mmchenry/Documents/Projects/waketracking'

# Matt on Linux
elif (platform.system() == 'Linux') and (os.path.expanduser('~')=='/home/mmchenry'):

    root_path = '/home/mmchenry/Documents/wake_tracking'

# Ashley on Linux
elif (platform.system() == 'Linux') and (os.path.expanduser('~')=='/home/anpetey'):

    root_path = '/vortex/schooling/TRex'

elif (platform.system() == 'Darwin') and (os.path.expanduser('~')=='/Users/experimentalist'):

    root_path = '/Volumes/schooling/TRex'

# Catch alternatives
else:
    raise ValueError('Do not recognize this account -- add lines of code to define paths here')

## Project paths

You can find instructions for defining hw_address in [README.md](README.md). You will need to be connected to the fileserver ("vortex") to access the log_path.

In [5]:
# Path to directory containing timecode audio file, needed if hardware is being controlled
if control_hw:
    # Path to time-code audio file
    aud_path = "/Users/experimentalist/Documents/Projects/wake_tracking/timecode.wav"

    # Check address of audio file
    if not os.path.isfile(aud_path):
        raise OSError("aud_path not found at " + aud_path)    
else:
    aud_path = None

# Create subdirectories for the project, if not present
paths = dp.give_paths(root_path, proj_dir)

# Path to log csv file
log_path = root_path  + os.path.sep + 'data' + os.path.sep + proj_dir + os.path.sep + 'recording_log.csv'

# Path to directory containing experiment schedules
schedules_path = root_path  + os.path.sep + 'data' + os.path.sep + proj_dir + os.path.sep + 'experiment_schedules'

# Check address of log file & scheudle directory
if not os.path.isfile(log_path):
    raise OSError("log_path not found at " + log_path)
if not os.path.isdir(schedules_path):
    raise OSError("schedules_path not found at " + schedule_path)

# log = pd.read_csv(log_path)
# print(log)

## Initialize hardware

Run these lines for all of the programs that you want to run below, even if you are not connected to the hardware.

In [6]:
if control_hw:
    #  Initialize hardware
    dmx = Controller(hw_address, auto_submit=True, dmx_size=32)

    # Set output rate at maximum
    dmx.set_dmx_parameters(output_rate=0)
    
else:
    dmx = None

    print('Harware control is off - in Debug mode')

Harware control is off - in Debug mode


# Basic hardware control
Useful for fiddling with the setup. You can skip this, if running experiments on a program or schedule.

## Change fixed light intensity
Note that if the Luxli LED lamp becomes unresponsive, then disconnect the DMX connection at the Enttec USB box, then power off the power supply to the light, turn the power supply back on, then power up the light (it should show the interactive screen), and then reconnect the DMX.

In [9]:
# Specify light intensity (from 0 to 1)
light_intensity = 0.5

# Sets DMX channel 1 to max 255 (Channel 1 is the intensity)
dmx.set_channel(1, int(light_intensity*255))  

## Turn on LED array

In [10]:
os.system('kasa --host ' + LED_IP + ' on')

No --type defined, discovering..
Turning on IR_LEDs


0

## Turn off LED array

In [9]:
os.system('kasa --host ' + LED_IP + ' off')

No --type defined, discovering..
Turning off IR_LEDs


0

## Take Calibration Video
Take a single calibration video with calibration stick


In [16]:
# Take number on video file for first recording
take_num_start = 100

# What light level to return to after the program
end_light_level = 0.5

# list of light levels
light_level = np.array([0.5])

# Duration that each level is fixed (min)
light_dur = np.array([1/60])


# Run control program
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, trig_video=True, echo=False, plot_data=False, 
    movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw,
    take_num=take_num_start,analyze_prompt=False)

if control_hw:
    # Set ending log intensity
    dmx.set_channel(1, int(end_light_level*255))  

print('calibration video completed')

    Video WILL be analyzed
    Video filename: SC_RN_BG_S001_S001_T100
    Log file saved to: /Volumes/schooling/TRex/data/miniScaleRN/recording_log.csv
    Trial 0 complete!
calibration video completed


# Single-Experiment Programs

Select which of the cells below that make sense for the experiment that you want to run.

## Lights on for fixed duration

In [ ]:
# What light level to return to after the program
end_light_level = 0.5

# list of light levels
light_level = np.array([1])

# Duration that each level is fixed (min)
light_dur = np.array([5/60])

# Run control program
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, trig_video=True, echo=False, plot_data=True, 
    movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw)

if control_hw:
    # Set ending log intensity
    dmx.set_channel(1, int(end_light_level*255))  

# print(df)

## Lights on -> Lights off

Ramp down light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([1, 0])

# Duration that each level is fixed (s)
light_dur = np.array([10, 10])

# Duration for the changes in light level
ramp_dur = np.array([0.5])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
    movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw)

# print(df)

## Lights off -> Lights on

Ramp down light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([0, 1])

# Duration that each level is fixed
light_dur = np.array([5, 5])

# Duration for the changes in light level
ramp_dur = np.array([2])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
    movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw)

# print(df)

## Ramp lights down and then up

Changing light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([1, 0, 0.5])

# Duration that each level is fixed
light_dur = np.array([1, 2, 1])

# Duration for the changes in light level
ramp_dur = np.array([0.5, 0.5])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
     movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw)

# print(df)

# Sequence of experiments

Using re.make_schedule, set change_var to 'ramp_dur_sec', 'start_dur_min', or 'end_dur_min' to vary an individual aspect of a lighting experiment. Then, set num_expts, min_val and max_val to the range of values for that variable among the experiments. Once the schedule file has been created, then use re.run_experiment_schedule to run the experiments. Note that the schedule is specific to a particular date and that the experiments should be run on that particular date. it is therefore easiest to create the schedule file on the same day as the running of the experiments.

## Make schedule for experiments at a fixed light intensity

In [7]:
# Duration of ramp before and after experiment (s)
pre_ramp_dur_sec = 1
post_ramp_dur_sec = 3

# Delay before starting experiment (min)
start_delay_min = 1

# list of starting and ending light levels
light_level = 0.5

# Light intensity between experiments
light_btwn = 0.5

# Duration that each level is fixed at the start and end of experiment (min)
dur_min = 5

# Number of experiments to run
num_expts = 3

# Period of time bwteen expts (min)
btwn_period_min = 10

# Write schedule to file
sch_file = re.make_schedule(paths['sch'], light_start=light_level, 
                            light_btwn=light_btwn, start_dur_min=dur_min, num_trial=num_expts,
                            btwn_period_min=btwn_period_min, pre_ramp_dur_sec=pre_ramp_dur_sec, post_ramp_dur_sec=post_ramp_dur_sec, start_delay_min=start_delay_min)

NOTE: Creating schedule with no ramps
--------------------------------------------------
Schedule file created: /Volumes/schooling/TRex/data/miniScaleRN/experiment_schedules/2023-06-07_sch01.csv
--------------------------------------------------
 


## Make schedule for experiments that vary ramp duration

A similar structure to the following code could be used where change_var is set to 'start_dur_min', or 'end_dur_min'.

In [ ]:
# Variable to change across experiments
change_var = 'ramp_dur_sec'

# Duration of ramp before and after experiment (s)
pre_ramp_dur_sec = 1
post_ramp_dur_sec = 3

# Delay before starting experiment (min)
start_delay_min = 3/60

# list of starting and ending light levels
light_start = 0.5
light_end   = 0.5

# Light intensity between experiments
light_btwn = 0.5

# Duration that each level is fixed at the start and end of experiment (min)
start_dur_min = 4/60
end_dur_min   = 6/60

# Duration for the ramp of changes in light level (s)
min_rampdur_sec = 5
max_rampdur_sec = 5

# Number of experiments to run
num_expts = 4

# Period of time bwteen expts (min)
btwn_period_min = .5


# Write schedule to file
sch_file = re.make_schedule(paths['sch'], change_var=change_var,  light_start=light_start, 
                            light_end=light_end, light_btwn=light_btwn, start_dur_min=start_dur_min, end_dur_min=end_dur_min, 
                            min_val=min_rampdur_sec, max_val=max_rampdur_sec, num_trial=num_expts, btwn_period_min=btwn_period_min, pre_ramp_dur_sec=pre_ramp_dur_sec, post_ramp_dur_sec=post_ramp_dur_sec, start_delay_min=start_delay_min)

## Execute experiments on a schedule

In [8]:
# Set this path to the schedule file you want to run
sch_file = paths['sch'] + os.sep + '2023-06-07_sch01.csv'

# Take number on video file for first recording
take_num_start = 5

# Execute experiments from schedule
re.run_experiment_schedule(dmx, aud_path=aud_path, log_path=log_path, 
    schedule_path=sch_file, movie_prefix=moviefile_prefix, LED_IP=LED_IP, 
    control_hw=control_hw, take_num_start=take_num_start)

Starting trial 1 at 2023-06-07, 14:46:46
    Starting audio to trigger video recording
    Timecode audio ended.
    Video WILL be analyzed
    Video filename: SC_RN_BG_S001_S001_T005
    Log file saved to: /Volumes/schooling/TRex/data/miniScaleRN/recording_log.csv
    Trial 1 complete!
Starting trial 2 at 2023-06-07, 15:01:46
    Starting audio to trigger video recording
    Timecode audio ended.
    Video WILL be analyzed
    Video filename: SC_RN_BG_S001_S001_T006
    Log file saved to: /Volumes/schooling/TRex/data/miniScaleRN/recording_log.csv
    Trial 2 complete!
Starting trial 3 at 2023-06-07, 15:16:46
    Starting audio to trigger video recording
    Timecode audio ended.
    Video WILL be analyzed
    Video filename: SC_RN_BG_S001_S001_T007
    Log file saved to: /Volumes/schooling/TRex/data/miniScaleRN/recording_log.csv
    Trial 3 complete!
